In [121]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch([{'host':'uct2-es-door.mwt2.org', 'port':9200}])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [234]:
ind="stashcp-2016.6"

myquery={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*a2gb*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1466622721000,
                  "lte": 1466674003000,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

In [309]:
page= es.search(index=ind, body=myquery, scroll='2m', search_type='scan', size=1)

sid = page['_scroll_id']

scroll_size = page['hits']['total']
results=[]

while (scroll_size > 0):
    #print ("Scrolling...")
    page = es.scroll(scroll_id = sid, scroll = '2m')
    results.append(page['hits']['hits'])
    # Update the scroll ID
    sid = page['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(page['hits']['hits'])
    #print ("scroll size: " + str(scroll_size))

scroll_size = page['hits']['total']
Res=[]
for i in range(0,scroll_size):
    Res.append(results[i][0]['_source'])
    
#print(scroll_size)

df = pd.DataFrame(Res)

del df['destination_space']
del df['xrdcp_version']
del df['tries']
del df['download_size']
del df['filesize']
del df['IP']
del df['xrdcp_exit']

In [340]:
df_UCSDT2=pd.DataFrame(columns=['download_time','filename','host','sitename','status','timestamp'])
for i in range(scroll_size):
    if df.iloc[i,3]=='UCSDT2-C':
        df_UCSDT2=df_UCSDT2.append(df.iloc[[i]])
        
df_UCSDT2=df_UCSDT2.sort_values(by=['timestamp'])
df_UCSDT2['try']=0
f=0
s=0
i=0
while i<200:
    df_UCSDT2.iloc[i,6]=1
    df_UCSDT2.iloc[i+1,6]=2
    f+=int(df_UCSDT2.iloc[i,0])
    s+=int(df_UCSDT2.iloc[i+1,0])
    i=i+2
    
#print(df_UCSDT2.shape[0])

#second / first try = speed-up
UCSDT2_speedup=(s/f)
print(UCSDT2_speedup)

200
0.078832089746585


In [343]:
def speed_up(sitename):
    df_site=pd.DataFrame(columns=['download_time','filename','host','sitename','status','timestamp'])
    for i in range(scroll_size):
        if df.iloc[i,3]==sitename:
            #print(df.iloc[[i]])
            df_site=df_site.append(df.iloc[[i]])
            
    df_site=df_site.sort_values(by=['timestamp'])
    df_site['try']=0
    f=0
    s=0
    i=0
    length=df_site.shape[0]
    if (length%2=1):
        
    while i<length:
        df_site.iloc[i,6]=1
        df_site.iloc[i+1,6]=2
        f+=int(df_site.iloc[i,0])
        s+=int(df_site.iloc[i+1,0])
        i+=2
    
    #print(df_site.head(100))

    #second / first try = speed-up
    speedup=(s/f)
    print (speedup)
    
#speed_up('UFlorida-HPC')

In [344]:
print("UCSDT2-C:")
speed_up('UCSDT2-C')
print("BNL_ATLAS_1:")
speed_up('BNL_ATLAS_1')
print("UFlorida-HPC:")
speed_up('UFlorida-HPC')
print("UNL (red-gateway2):")
speed_up('red-gateway2')
print("SU-OG-CE:")
speed_up('SU-OG-CE')
print("Sandhills-CE1:")
speed_up('Sandhills-CE1')
print("MWT2_CE_UIUC:")
speed_up('MWT2_CE_UIUC')


UCSDT2-C:
200
0.078832089746585
BNL_ATLAS_1:
200
0.9548963602338066
UFlorida-HPC:
200
0.9903454606879278
UNL (red-gateway2):
200
0.2435541355414621
SU-OG-CE:
193


IndexError: single positional indexer is out-of-bounds

In [347]:
a=7
print(a%2)

1
